[View in Colaboratory](https://colab.research.google.com/github/juzkev/A-Bridged/blob/master/Repo.ipynb)

In [1]:
# Time
import time
start_t = time.time()

# Google Drive
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#textract
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libpulse-dev
!pip install textract
import textract

# DS essentials
import pandas as pd
import numpy as np

# NLP
import re



Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
python-dev is already the newest version (2.7.14-2ubuntu1).
libxml2-dev is already the newest version (2.9.4+dfsg1-4ubuntu1.2).
The following additional packages will be installed:
  dbus fontconfig fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  i965-va-driver libaacs0 libapparmor1 libasound2 libasound2-data libass9
  libasyncns0 libavahi-client3 libavahi-common-data libavahi-common3
  libavc1394-0 libavcodec57 libavdevice57 libavfilter6 libavformat57
  libavresample3 libavutil55 libbdplus0 libbluray2 libboost-filesystem1.62.0
  libboost-system1.62.0 libbs2b0 libcaca0 libcairo2 libcapnp-0.5.3
  libcdio-cdda1 libcdio-paranoia1 libcdio13 libchromaprint1 libcrystalhd3
  libcups2 libcupsfilters1 libcupsimage2 libdatrie1 libdbus-1-3 libdc1394-22
  libdrm-amdgpu1 libdrm-common libdrm-intel1 

In [2]:
# Install Spacy required for fastai.text
!pip install -U spacy
!python -m spacy download en
import spacy

    100% |████████████████████████████████| 22.0MB 1.6MB/s 
    100% |████████████████████████████████| 122kB 26.0MB/s 
    100% |████████████████████████████████| 1.2MB 13.5MB/s 
    100% |████████████████████████████████| 194kB 24.4MB/s 
    100% |████████████████████████████████| 153kB 25.4MB/s 
    100% |████████████████████████████████| 604kB 13.6MB/s 
    100% |████████████████████████████████| 317kB 17.5MB/s 
    100% |████████████████████████████████| 450kB 17.3MB/s 
    100% |████████████████████████████████| 143kB 25.7MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/60/0b/b

# Downloading PDFs from Google Drive

In [15]:
import os

local_download_path = os.path.expanduser('~/data')

try:
  os.makedirs(local_download_path)
except: pass

file_list = drive.ListFile(
        {'q': f"'1SxZlizNbA6mvvLZUUYPTzam46K6G597B' in parents"}).GetList()

for f in file_list:
    fname = os.path.join(local_download_path, f['title'])
    print('downloading to {}'.format(fname))
    try:
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
    except:pass

downloading to /root/data/2016CMMT CSR report en.pdf
downloading to /root/data/44 Final Sustainability Report to CoSec 10Feb16.pdf
downloading to /root/data/17_0080 Cambridge SR2016_v5single_Final.pdf
downloading to /root/data/Wilmar Sustainability Report 2017.pdf
downloading to /root/data/Wilmar SR 2016 - Final.pdf
downloading to /root/data/Wilmar Policy Progress Report - Dec 2013 to Dec 2015.pdf
downloading to /root/data/Weiye Sustainability Report FY2017_ENG.pdf
downloading to /root/data/VIT 2017 Sustainability Report.pdf
downloading to /root/data/Venture Sustainability Report 2017.pdf
downloading to /root/data/Valuetronics Holdings Limited_Sustainability Report 2018.pdf
downloading to /root/data/ValueMax_20180531 -sustainability report.pdf
downloading to /root/data/UOL_Sustainability_Report_2017.pdf
downloading to /root/data/TMGL Sustainability Report 2017.pdf
downloading to /root/data/Tiong Seng Sustainability Report 2017.pdf
downloading to /root/data/TGL_FY17 Sustainability Repor

# Reading the PDF into DF

In [20]:
# Reading the PDF
df = pd.DataFrame()
succ = 0
fail = 0
# pattern = 'SDG (\d{2})'
s_t = time.time()

for filename in os.listdir(os.path.join(local_download_path)):
  try:
    print(os.path.join(local_download_path, filename))

    # Textract code
    t = textract.process(os.path.join(local_download_path, filename))

    df=df.append({'filename': filename, 'fulltext': t}, ignore_index=True)
    succ += 1
  except Exception as e: 
    print(e)
    fail += 1

print(f'''Successfully processed {succ}/{succ+fail} pdfs''')
print(f'Reading PDFs took {time.time()-s_t} seconds')

/root/data/Wilmar Policy Progress Report - Dec 2013 to Dec 2015.pdf
/root/data/ORI0086_Sust Report_FA1.pdf
/root/data/KCL_SR2017.pdf
/root/data/Brook Crompton Sustainability Report.pdf
/root/data/Wilmar_Fourth Quarter 2015 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/ASX015. Sustainable Development Report 2016.pdf
/root/data/Wilmar - Third Quarter 2015 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/Forise - SR Report 2017.pdf
/root/data/Wilmar-Third Quarter 2017 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/Wilmar- First Quarter 2015 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/UOL_Sustainability_Report_2017.pdf
/root/data/Accordia Golf Trust - Sustainability Report - 31 March 2018.pdf
/root/data/OUELH  20180521 2017 Sustainability Report.pdf
/root/data/Wilmar-Second Quarter 2017 Update Report on its No Deforestation No Peat No Exploit

In [21]:
df.head()

,filename,fulltext
0,Wilmar Policy Progress Report - Dec 2013 to De...,"b""NO DEFORESTATION,\nNO PEAT,\nNO EXPLOITATION..."
1,ORI0086_Sust Report_FA1.pdf,"b""WE HELP\nJESS HARNESS\nTHE SUN\xe2\x80\x99S\..."
2,KCL_SR2017.pdf,b'Solutions for\nSustainable Urbanisation\nSus...
3,Brook Crompton Sustainability Report.pdf,b'ABOUT THIS REPORT\nThis report is set out on...
4,Wilmar_Fourth Quarter 2015 Update Report on it...,"b'Wilmar\xe2\x80\x99s No Deforestation, No Pea..."


In [23]:
# Breaking each corpus down into sentences using spacy .sent property

nlp = spacy.load('en')
doc = nlp(df.iloc[0,1].decode())
for sent in doc.sents:
    print(sent.text)

NO DEFORESTATION,
NO PEAT,

NO EXPLOITATION

POLICY PROGRESS UPDATE
(DECEMBER 2013 - DECEMBER 2015)

Wilmar International Limited

CONTENTS

PAGE
3

Commentary by Glenn Hurowitz, Senior Fellow
– Centre for International Policy

4


Our Aspiration:
A Sustainable Palm Oil Industry

5


Wilmar's Key Policy Milestones

6

6
8
11
13

15
15
17


Our Progress in 2 Years

A Traceable Supply Chain

Overview of Wilmar's Supply Chain
Supplier Compliance to the Wilmar Policy

Wilmar's own compliance to the Policy

No Exploitation of Labour
Support for Smallholders

Risk Management: Transparency and Accountability

18
18
19
19


Changes in the Industry
Stakeholder Engagement
Multi-stakeholder Collaborations
The Landscape & Jurisdictional Approach

20
20
20
20
20
21
21

2016 and Beyond: New Targets and Facing the Challenges
Furthering Supplier Compliance through a Regional Landscape Approach
Traceability Back to Plantations
Empowering Smallholders
Identifying Labour Best Practices & Removing Exploi

In [28]:
# Using nltk punkt sentence tokenizer

from nltk.tokenize.punkt import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()
# tokenizer.train(df.fulltext.str.decode('utf-8'))

# Cleaning "\n" in between sentences as they are essentially meaningless
[w.replace('\n', ' ') for w in tokenizer.tokenize(df.iloc[0,1].decode())]

["NO DEFORESTATION, NO PEAT, NO EXPLOITATION POLICY PROGRESS UPDATE (DECEMBER 2013 - DECEMBER 2015) Wilmar International Limited  \x0cCONTENTS PAGE 3  Commentary by Glenn Hurowitz, Senior Fellow – Centre for International Policy  4  Our Aspiration: A Sustainable Palm Oil Industry  5  Wilmar's Key Policy Milestones  6 6 8 11 13 15 15 17  Our Progress in 2 Years A Traceable Supply Chain Overview of Wilmar's Supply Chain Supplier Compliance to the Wilmar Policy Wilmar's own compliance to the Policy No Exploitation of Labour Support for Smallholders Risk Management: Transparency and Accountability  18 18 19 19  Changes in the Industry Stakeholder Engagement Multi-stakeholder Collaborations The Landscape & Jurisdictional Approach  20 20 20 20 20 21 21  2016 and Beyond: New Targets and Facing the Challenges Furthering Supplier Compliance through a Regional Landscape Approach Traceability Back to Plantations Empowering Smallholders Identifying Labour Best Practices & Removing Exploitation Red